In [1]:
import pandas as pd
import numpy as np

import regex as re
import altair as alt
alt.data_transformers.disable_max_rows()

from datetime import datetime
from datetime import timedelta

In [ ]:
"I am not entirely sure what this file is, or if it is important. It looks like \
Vicens exploring the data (?)"

In [22]:
'/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/code/CEWS_v1_0-eng/'
'/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/'

# Databases loading
cews = pd.read_csv("/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/code/CEWS_v1_0-eng/data/CEWS_with_subset_labels.csv")

## This dataset is taken from StatCan and available in our repo (https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1410037701).
re_counts = pd.read_csv("/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/more_data/RuralEmploymentIndustry.csv")
ue_counts = pd.read_csv("/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/more_data/UrbanEmploymentIndustry.csv")
e_counts = pd.read_csv("/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/more_data/businessCountByWorkerIndustry.csv")
a_counts = pd.read_csv("/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/more_data/AlbertabusinessCountByWorkerIndustry.csv")

In [23]:
# Changing data types
cews = cews.drop("Unnamed: 0", axis = 1)
cews.Period = pd.to_datetime(cews.Period, infer_datetime_format=True)

# Indicating intereset varaibles for analysis
num_cols = ['BusinessLocations', 'Subsidy', 'SupportedEmployees', 'RehiredEmployees']
provinces = ['10-rural', '11-rural', '12-rural', '13-rural', '24-rural', '35-rural', '46-rural', '47-rural', '48-rural', '59-rural']
provinces_ur = ['10-urban', '11-urban', '12-urban', '13-urban', '24-urban', '35-urban', '46-urban', '47-urban', '48-urban', '59-urban']
territories = ['60-rural', '61-rural', '62-rural']
industries = ['Construction', 'Manufacturing', 'Accommodation and food services', 'Mining, quarrying, and oil and gas extraction']

alberta = cews[cews.RegionCode.str.startswith('48')]

In [58]:
aaa = (alberta.query("IndustryCode == '21' or IndustryCode == '113' or IndustryCode == '114' or IndustryCode == '23' or IndustryCode == '72' or IndustryCode == '31-33'"
             )).query("GeoAggregation == 'Province'")

aaa[aaa.isnull().any(axis=1)]

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation
52315,2020-03-15,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2
128438,2020-04-12,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2
205826,2020-05-10,48,Alberta,Not applicable,Not applicable,113,Forestry and logging,50.0,NaN,539.0,0.0,Province,Level 2
205827,2020-05-10,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2
278591,2020-06-07,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2
420598,2020-08-02,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2
486837,2020-08-30,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2
547169,2020-09-27,48,Alberta,Not applicable,Not applicable,113,Forestry and logging,25.0,NaN,556.0,5.0,Province,Level 2
547170,2020-09-27,48,Alberta,Not applicable,Not applicable,114,"Fishing, hunting and trapping",0.0,NaN,NaN,0.0,Province,Level 2


In [14]:
a_counts.rename(columns={'Population centre and rural areas': 'GeographicClassification', 'Class of worker': 'Class', 
                         'North American Industry Classification System (NAICS)' : 'Industry', 'VALUE' : 'Employees'}, inplace=True)
dummy = a_counts.Industry.str.split('[', n=1, expand=True)
a_counts['IndustryCode'] = dummy[1].str.split(']').str[0]
a_counts['Employees'] = a_counts['Employees'] * 1000
a_counts.Industry = dummy[0].str.rstrip()
a_counts.head(10)

,GeographicClassification,Class,Industry,Employees,IndustryCode
0,Census metropolitan area and census agglomeration,"Total employed, all classes of workers","Total employed, all industries",1827800.0,None
1,Census metropolitan area and census agglomeration,"Total employed, all classes of workers",Agriculture,12200.0,"111-112, 1100, 1151-1152"
2,Census metropolitan area and census agglomeration,"Total employed, all classes of workers","Forestry, fishing, mining, quarrying, oil and gas",104900.0,"21, 113-114, 1153, 2100"
3,Census metropolitan area and census agglomeration,"Total employed, all classes of workers",Construction,183500.0,23
4,Census metropolitan area and census agglomeration,"Total employed, all classes of workers",Manufacturing,103900.0,31-33
5,Census metropolitan area and census agglomeration,"Total employed, all classes of workers",Accommodation and food services,100200.0,72
6,Census metropolitan area and census agglomeration,Employees,"Total employed, all industries",1555700.0,None
7,Census metropolitan area and census agglomeration,Employees,Agriculture,7300.0,"111-112, 1100, 1151-1152"
8,Census metropolitan area and census agglomeration,Employees,"Forestry, fishing, mining, quarrying, oil and gas",98800.0,"21, 113-114, 1153, 2100"
9,Census metropolitan area and census agglomeration,Employees,Construction,139600.0,23


In [ ]:
re_counts.rename(columns={'GEO': 'Region', 'North American Industry Classification System (NAICS)' : 'Industry', 'VALUE' : 'Total Employees'}, inplace=True)
dummy = re_counts.Industry.str.split('[', n=1, expand=True)
re_counts['IndustryCode'] = dummy[1].str.split(']').str[0]
re_counts['Total Employees'] = re_counts['Total Employees'] * 1000
re_counts.Industry = dummy[0].str.rstrip()